# Find all Tuesdays

In this tutorial we look at two ways to solve the following problem:

> Count the number of Tuesday between 1st of January 2000 and 31st of December 2020

First, we find the answer using the very helpful [Datetime](https://docs.python.org/3/library/datetime.html) package. This package does most of the heavy lifting for us. Then, for a bit of fun, we try and find the same solution using Python's default data types only. 

## 1. With the datetime package

Start by importing the datetime and timedelta class.

In [1]:
from datetime import datetime, timedelta

Then write a function which takes two paraters, `start_year` and `end_year`, which returns the number of Tuesdays between these years. There is an assumption that `start_year` starts on 1st day of the year, and `end_year` ends on the last day of that year.

In [2]:
def count_tuesdays(start_year, end_year):
    
    """
    Returns the number of Tuesaday between two dates.
    
    Parameters:
        start_year (int): The starting year
        end_year (int): The end year
        
    Returns:
        count_tuesdays(int): The number of Tuesdays between start_year and end_year
    
    """
    
    # 1. Find the number of days between start and end year
    datetime_range =  datetime(year=end_year, month=1, day=1) - datetime(year=start_year, month=12, day=31)
    
    # 2. Build a list of Datetimes for this range
    date_range = [datetime(year=start_year, month=1, day=1) + timedelta(days=x) for x in range(datetime_range.days)]
        
    # 3. Count the number of Tuesdays in this range    
    count_tuesdays = len([i for i in date_range if i.isoweekday() == 2])
        
    return count_tuesdays

In [3]:
count_tuesdays(2000, 2021)

1044

## 2. Using default Python data structures

Next, lets make it a little harder to solve. Lets find the same result, this time only using Python's default data structures. In order words, we cannot rely on the `datetime` package. 

First, we create a simple `Date` object which has the following attributes:
    
1. year
2. month
3. day
4. is_leap
5. day_of_week
6. day_of_weel_decode

There is nothing too complicated here. We assign some validation on the attributes. For example, the days cannot be greater than 31 and months cannot be greater than 12.

In [4]:
class Date():
    
    def __init__(self, year, month, day, day_of_week):
        self.year = year
        self.is_leap = self._is_leap(self.year)
        self.month = month
        self.day = day
        self.day_of_week = day_of_week
        self.day_of_week_decode = self._day_of_week_decode()
        
    @property
    def day(self):
        return self._day
    
    @day.setter
    def day(self, value):
        if (value > 31):
            raise Exception("Days cannot be greater than 31") 
        if (self.month in [9, 4, 6, 11] and value > 31):
            raise Exception(f"Days cannot be greater than 30 for month {self.month}") 
        if (self.month == 2):
            if (self.is_leap == True and value > 29):
                raise Exception(f"Days must be 29 or less for a leap year Febuary")
            if (self.is_leap == False and value > 28):
                raise Exception(f"Days must be 28 or less for a non-leap year Febuary")        
        self._day = value
        
    @property
    def month(self):
        return self._month
    
    @month.setter
    def month(self, value):
        if (value > 12): raise Exception("Month cannot be greater than 12") 
        self._month = value
    
    @property
    def year(self):
        return self._year

    @year.setter
    def year(self, value):
        self._year = value
        
    @property
    def day_of_week(self):
        return self._day_of_week

    @day_of_week.setter
    def day_of_week(self, value):
        self._day_of_week = value
        
    def _is_leap(self, year):
        
        if (year % 4 == 0) & (year % 100 != 0):
            return True

        if (year % 100 == 0) & (year % 400 == 0):
            return True

        return False
        
    def _day_of_week_decode(self):
        
        decodes = {
            
            1: "Mon",
            2: "Tue",
            3: "Wed",
            4: "Thu",
            5: "Fri",
            6: "Sat",
            7: "Sun"
            
        }
        
        return decodes[self.day_of_week]
        
    def __str__(self):
        return f"{self.year}-{self.month}-{self.day}"
    
    def __repr__(self):
        return f"Date({self.year}-{self.month}-{self.day}, {self.day_of_week_decode})"
    
        

Let's build a sample date object to check everything looks ok. We'll create a `Date` for Tuesday 29th of Febuary 2000.

In [5]:
# Create a valid object

Date(2000, 2, 29, 2)

Date(2000-2-29, Tue)

In [6]:
# Attempt to create an invalid object

try:
    sample_date =  Date(2001, 2, 29, 2)
except Exception as e:
    print(e)

Days must be 28 or less for a non-leap year Febuary


Next, let's build up a range of dates. First we need a generator which will return a weekday label (1 to 7) for each date in our range. We know that the 1st of January is a saturday so we will build our iterable from that point. 

In [7]:
class DateRange():
    
    def __init__(self, start_year, end_year, start_day_of_week):
        self.start_year = start_year
        self.end_year = end_year
        self.start_day_of_week = start_day_of_week
        self.range = self.build_range(start_year, end_year, start_day_of_week)
        
    def _days_of_week(self, starting_day):

        days_of_week = list(range(1,8))

        days_of_week = days_of_week[(starting_day-1):] + days_of_week[:(starting_day-1)]

        while True:

            for n in days_of_week:

                yield(n)
                
    def _is_leap(self, year):
        
        if (year % 4 == 0) & (year % 100 != 0):
            return True

        if (year % 100 == 0) & (year % 400 == 0):
            return True

        return False

    def build_range(self, start_year, end_year, start_day_of_week):
                
        day_of_week_iterator = self._days_of_week(start_day_of_week)
        
        date_range = []

        for year in range(start_year, end_year+1):

            for month in range(1, 13):

                if month in [9, 4, 6, 11]:
                    day_count = 30

                elif (month == 2 and self._is_leap(year)):
                    day_count = 29

                elif (month == 2 and self._is_leap(year) is False):
                    day_count = 28

                else:
                    day_count = 31

                days_in_month = [Date(year, month, i, next(day_of_week_iterator)) for i in range(1, day_count+1)]

                date_range.extend(days_in_month)

        return date_range

In [8]:
date_range = DateRange(2000, 2019, 6).range
count_tuesdays = len([i for i in date_range if i.day_of_week == 2])
count_tuesdays

1044